In [1]:
import torch
import mcunet

from mcunet.model_zoo import build_model, net_id_list, download_tflite, download_url
from mcunet.utils import AverageMeter, accuracy, count_net_flops, count_parameters
from torchinfo import summary

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, img_size, description = build_model(net_id='mcunet-in4', pretrained=True)


In [3]:
print(img_size)
print(description)

160
MCUNet model that fits 512KB SRAM and 2MB Flash (ImageNet)


In [16]:
summary(model, input_size=(1, 3, img_size, img_size), col_width=30, col_names=['kernel_size', 'output_size', 'num_params', 'mult_adds', 'params_percent'], depth=8)

Layer (type:depth-idx)                        Kernel Shape                   Output Shape                   Param #                        Mult-Adds                      Param %
ProxylessNASNets                              --                             [1, 1000]                      --                             --                                  --
├─ConvLayer: 1-1                              3                              [1, 32, 80, 80]                --                             --                                  --
│    └─Conv2d: 2-1                            [3, 3]                         [1, 32, 80, 80]                864                            5,529,600                        0.05%
│    └─BatchNorm2d: 2-2                       --                             [1, 32, 80, 80]                64                             64                               0.00%
│    └─ReLU6: 2-3                             --                             [1, 32, 80, 80]                --

In [17]:
output = summary(model, input_size=(1, 3, img_size, img_size), col_width=16, col_names=['kernel_size', 'output_size', 'num_params', 'mult_adds', 'params_percent'], depth=6)

In [18]:
text_output = output.__repr__()

In [19]:
with open('mcunet-in4.txt', 'w') as f:
    print(text_output, file=f)

new_txt_line = []
with open('mcunet-in4.txt', 'r') as f:
    for line in f.readlines():
        if 'ReLU6' in line or 'BatchNorm2d' in line or 'ReLU' in line:
            continue
        else:
            new_txt_line.append(line)
conv_result = ''.join(new_txt_line)
print(conv_result)

Layer (type:depth-idx)                        Kernel Shape     Output Shape     Param #          Mult-Adds        Param %
ProxylessNASNets                              --               [1, 1000]        --               --                    --
├─ConvLayer: 1-1                              3                [1, 32, 80, 80]  --               --                    --
│    └─Conv2d: 2-1                            [3, 3]           [1, 32, 80, 80]  864              5,529,600          0.05%
├─ModuleList: 1-2                             --               --               --               --                    --
│    └─MobileInvertedResidualBlock: 2-4       --               [1, 16, 80, 80]  --               --                    --
│    │    └─MBInvertedConvLayer: 3-1          3                [1, 16, 80, 80]  --               --                    --
│    │    │    └─Sequential: 4-1              --               [1, 32, 80, 80]  --               --                    --
│    │    │    │    └─Co

In [26]:
new_txt_line = []
with open('mcunet-in4.txt', 'r') as f:
    for line in f.readlines():
        if 'ReLU6' in line or 'BatchNorm2d' in line or 'ReLU' in line or '--' in line[-5:]:
            continue
        else:
            new_txt_line.append(line)
conv_result = ''.join(new_txt_line)
print(conv_result)

# compute the total params
param_list = []
for line in new_txt_line:
    if 'Conv2d' in line or 'Linear' in line:
        param_list.append(float(line.split('%')[0][-6:]))
        


Layer (type:depth-idx)                        Kernel Shape     Output Shape     Param #          Mult-Adds        Param %
│    └─Conv2d: 2-1                            [3, 3]           [1, 32, 80, 80]  864              5,529,600          0.05%
│    │    │    │    └─Conv2d: 5-1             [3, 3]           [1, 32, 80, 80]  288              1,843,200          0.02%
│    │    │    │    └─Conv2d: 5-4             [1, 1]           [1, 16, 80, 80]  512              3,276,800          0.03%
│    │    │    │    └─Conv2d: 5-6             [1, 1]           [1, 48, 80, 80]  768              4,915,200          0.04%
│    │    │    │    └─Conv2d: 5-9             [7, 7]           [1, 48, 40, 40]  2,352            3,763,200          0.14%
│    │    │    │    └─Conv2d: 5-12            [1, 1]           [1, 24, 40, 40]  1,152            1,843,200          0.07%
│    │    │    │    └─Conv2d: 5-14            [1, 1]           [1, 120, 40, 40] 2,880            4,608,000          0.17%
│    │    │    │    └─Co

In [57]:
new_txt_line = []
with open('mcunet-in4.txt', 'r') as f:
    for line in f.readlines():
        if 'ReLU6' in line or 'BatchNorm2d' in line or 'ReLU' in line or '--' in line[-5:]:
            continue
        else:
            new_txt_line.append(line)
conv_result = ''.join(new_txt_line)
print(conv_result)

# depthwise vs pointwise
pw_param_list = []
dw_param_list = []
for line in new_txt_line:
    if 'Conv2d' in line and '[1, 1]' in line:
        pw_param_list.append(float(line.split('%')[0][-6:]))
    elif 'Conv2d' in line:
        dw_param_list.append(float(line.split('%')[0][-6:]))

Layer (type:depth-idx)                        Kernel Shape     Output Shape     Param #          Mult-Adds        Param %
│    └─Conv2d: 2-1                            [3, 3]           [1, 32, 80, 80]  864              5,529,600          0.05%
│    │    │    │    └─Conv2d: 5-1             [3, 3]           [1, 32, 80, 80]  288              1,843,200          0.02%
│    │    │    │    └─Conv2d: 5-4             [1, 1]           [1, 16, 80, 80]  512              3,276,800          0.03%
│    │    │    │    └─Conv2d: 5-6             [1, 1]           [1, 48, 80, 80]  768              4,915,200          0.04%
│    │    │    │    └─Conv2d: 5-9             [7, 7]           [1, 48, 40, 40]  2,352            3,763,200          0.14%
│    │    │    │    └─Conv2d: 5-12            [1, 1]           [1, 24, 40, 40]  1,152            1,843,200          0.07%
│    │    │    │    └─Conv2d: 5-14            [1, 1]           [1, 120, 40, 40] 2,880            4,608,000          0.17%
│    │    │    │    └─Co

In [60]:
print(np.array(pw_param_list))

[0.030 0.040 0.070 0.170 0.170 0.130 0.130 0.170 0.280 0.370 0.370 0.370
 0.370 0.280 0.550 1.110 1.110 1.110 1.110 1.480 1.770 1.600 1.600 1.600
 1.600 2.130 4.260 6.380 6.380 6.380 6.380 8.510 14.180]


In [61]:
print(np.array(dw_param_list))

[0.050 0.020 0.140 0.060 0.140 0.340 0.080 0.450 0.340 0.120 0.680 0.170
 0.420 0.420 1.090 1.630 0.830 1.110]


In [34]:
import numpy as np
np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.3f}".format(x)})
print(np.cumsum(param_list))

[0.050 0.070 0.100 0.140 0.280 0.350 0.520 0.580 0.750 0.880 1.020 1.150
 1.320 1.660 1.940 2.310 2.390 2.760 3.130 3.580 3.950 4.230 4.570 5.120
 6.230 6.350 7.460 8.570 9.250 10.360 11.840 12.010 13.780 15.380 15.800
 17.400 19.000 19.420 21.020 23.150 24.240 28.500 34.880 36.510 42.890
 49.270 50.100 56.480 64.990 66.100 80.280 98.810]


In [45]:
output = summary(model, input_size=(1, 3, img_size, img_size), col_width=16, col_names=['kernel_size', 'output_size', 'num_params', 'mult_adds', 'params_percent'], depth=2)
print(output)

Layer (type:depth-idx)                        Kernel Shape     Output Shape     Param #          Mult-Adds        Param %
ProxylessNASNets                              --               [1, 1000]        --               --                    --
├─ConvLayer: 1-1                              3                [1, 32, 80, 80]  --               --                    --
│    └─Conv2d: 2-1                            [3, 3]           [1, 32, 80, 80]  864              5,529,600          0.05%
│    └─BatchNorm2d: 2-2                       --               [1, 32, 80, 80]  64               64                 0.00%
│    └─ReLU6: 2-3                             --               [1, 32, 80, 80]  --               --                    --
├─ModuleList: 1-2                             --               --               --               --                    --
│    └─MobileInvertedResidualBlock: 2-4       --               [1, 16, 80, 80]  896              5,120,096          0.05%
│    └─MobileInvertedRes

In [46]:
import os
def get_txt_result(model, img_size, depth=6):
    output = summary(model, input_size=(1, 3, img_size, img_size), col_width=16, col_names=['kernel_size', 'output_size', 'num_params', 'mult_adds', 'params_percent'], depth=depth)
    dummy_name = str(np.random.randn(1)) + '.txt'
    # save summary output to dummy file
    with open(dummy_name, 'w') as f:
        print(output, file=f)

    # load summary output
    new_txt_line = []
    with open(dummy_name, 'r') as f:
        for line in f.readlines():
            if 'ReLU6' in line or 'BatchNorm2d' in line or 'ReLU' in line:
                continue
            else:
                new_txt_line.append(line)
    conv_result = ''.join(new_txt_line)
    os.remove(dummy_name)
    return conv_result, new_txt_line
    

In [49]:
conv_result, new_txt_line = get_txt_result(model, img_size, depth=2)
print(conv_result)

Layer (type:depth-idx)                        Kernel Shape     Output Shape     Param #          Mult-Adds        Param %
ProxylessNASNets                              --               [1, 1000]        --               --                    --
├─ConvLayer: 1-1                              3                [1, 32, 80, 80]  --               --                    --
│    └─Conv2d: 2-1                            [3, 3]           [1, 32, 80, 80]  864              5,529,600          0.05%
├─ModuleList: 1-2                             --               --               --               --                    --
│    └─MobileInvertedResidualBlock: 2-4       --               [1, 16, 80, 80]  896              5,120,096          0.05%
│    └─MobileInvertedResidualBlock: 2-5       --               [1, 24, 40, 40]  4,512            10,521,840         0.26%
│    └─MobileInvertedResidualBlock: 2-6       --               [1, 24, 40, 40]  7,368            10,944,528         0.43%
│    └─MobileInvertedRes

In [62]:
print(model)

ProxylessNASNets(
  (first_conv): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act): ReLU6(inplace=True)
  )
  (blocks): ModuleList(
    (0): MobileInvertedResidualBlock(
      (mobile_inverted_conv): MBInvertedConvLayer(
        (depth_conv): Sequential(
          (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU6(inplace=True)
        )
        (point_linear): Sequential(
          (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
    )
    (1): MobileInvertedResidualBlock(
      (mobile_inverted_conv): MBInvertedConvLayer(
        (inverted_b

In [54]:
mb_layer_result=[]
for line in new_txt_line:
    if 'Conv2d' in line or 'Linear' in line or 'MobileInvertedResidualBlock' in line:
        if '--' not in line.split('%')[0][-6:]:
            mb_layer_result.append(float(line.split('%')[0][-6:]))
                                   
np_mb_layer = np.array(mb_layer_result)

print(len(np_mb_layer))
print(np_mb_layer)
print(np.cumsum(np_mb_layer))


19
[0.050 0.050 0.260 0.430 0.430 0.810 0.860 1.230 1.210 2.410 2.960 3.500
 3.680 3.680 7.580 14.550 13.750 24.020 18.530]
[0.050 0.100 0.360 0.790 1.220 2.030 2.890 4.120 5.330 7.740 10.700 14.200
 17.880 21.560 29.140 43.690 57.440 81.460 99.990]


### MCUNet-in2 Check

In [14]:
model2, img_size, description = build_model(net_id='mcunet-in2', pretrained=True)

In [15]:
print(img_size)
print(description)

160
MCUNet model that fits 256KB SRAM and 1MB Flash (ImageNet)


In [16]:
len(model2.blocks)

18

In [17]:
summary(model2, input_size=(1, 3, img_size, img_size), col_width=16, col_names=['kernel_size', 'output_size', 'num_params', 'mult_adds', 'params_percent'])

Layer (type:depth-idx)                        Kernel Shape     Output Shape     Param #          Mult-Adds        Param %
ProxylessNASNets                              --               [1, 1000]        --               --                    --
├─ConvLayer: 1-1                              3                [1, 16, 80, 80]  --               --                    --
│    └─Conv2d: 2-1                            [3, 3]           [1, 16, 80, 80]  432              2,764,800          0.06%
│    └─BatchNorm2d: 2-2                       --               [1, 16, 80, 80]  32               32                 0.00%
│    └─ReLU6: 2-3                             --               [1, 16, 80, 80]  --               --                    --
├─ModuleList: 1-2                             --               --               --               --                    --
│    └─MobileInvertedResidualBlock: 2-4       --               [1, 8, 80, 80]   --               --                    --
│    │    └─MBInvertedCo